In [1]:
import acquire, prepare, model

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
import pandas as pd

In [2]:
df = acquire.get_corpus()

df['clean_and_lem'] = df.readme.apply(prepare.basic_clean).apply(prepare.lemmatize)

df = prepare.cut_singles(df)

In [ ]:
#HOLD until we try it without it
##
#df['encoded_language'] = model.encode_categories(df.language)

In [5]:
X_train, X_test, train, test = model.make_model_components(df.clean_and_lem, df.language, .15)

In [9]:
#####THE CLASSIFIER!
tree = DecisionTreeClassifier(max_depth=6).fit(X_train, train.actual)

In [4]:
predictons = model.run_the_tree_model(X_train, train.actual, 6)

In [ ]:
train['predictions'] = predictons

In [ ]:
model.score_your_model(train.actual, train.predictions)

In [ ]:
test_predictons = model.run_the_tree_model(X_test, test.actual, 6)
test['predictions'] = test_predictons

In [ ]:
model.score_your_model(test.actual, test.predictions)

In [ ]:
df2 = acquire.get_corpus()

df2['clean-lem-no_stop'] = df2.readme.apply(prepare.basic_clean).apply(prepare.remove_stopwords).apply(prepare.lemmatize)

df2 = prepare.cut_singles(df2)

In [ ]:
df2.head()

In [ ]:
X_train, X_test, train, test = model.make_model_components(df2['clean-lem-no_stop'], df2.language)

In [ ]:
predictons2 = model.run_the_tree_model(X_train, train.actual, 6)

In [ ]:
train['predictions'] = predictons2

In [ ]:
model.score_your_model(train.actual, train.predictions)

In [ ]:
test_predictons2 = model.run_the_tree_model(X_test, test.actual, 6)
test['predictions'] = test_predictons2

In [ ]:
model.score_your_model(test.actual, test.predictions)

# MAKING 2nd Corpus

In [ ]:
from requests import get
from bs4 import BeautifulSoup

import pandas as pd

import json
import os

In [ ]:
def scrape_links_one_page(url):
    headers = {'User-Agent': 'Codeup Data Science'} 
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    extension = soup.find_all("a", {"class":"v-align-middle"}, {"data-hydro-click":"url"})
    extensions = []
    for count in range(len(extension)):
        extensions.append("http://github.com/" + extension[count].get_text())
    return extensions

def make_url_list():
    """
    Creates a 10 page list. Each page needs the links to repositories scapes.
    """
    urls = []
    for count in range(11,21):
        urls.append("https://github.com/search?p={}&q=stars%3A%3E100&type=Repositories".format(count))
    return urls


def loop_through_urls(url_list):
    """
    Creates a 100 page list. Each element is a link to be scraped of their README.
    """
    big_list = []
    for url in url_list:
        big_list.extend(scrape_links_one_page(url))
    return big_list

def get_readme_text(url):
    response = get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    textboxes = soup.findAll("div", {"class": "Box-body"})
    readme_text = textboxes[0].text

    language = soup.find("span", {"class": "lang"})
    if language == None:
        language = "None"
    else:
        language = soup.find("span", {"class": "lang"}).get_text()

    return {
        "repo" : url[18:],
        "language" : language,
        "readme" : readme_text
    }


In [ ]:
url_list = make_url_list()

In [ ]:
url_list

In [ ]:
big_list = loop_through_urls(url_list)

In [ ]:
pages10_20 = pd.Series(big_list)

In [ ]:
pages10_20

In [ ]:
def make_corpus(big_list):
    corpus = [get_readme_text(url) for url in big_list]
    return corpus

In [ ]:
corpus2 = make_corpus(big_list)

In [ ]:
df_corpus2 = pd.DataFrame(corpus2)

In [ ]:
df_corpus2

In [ ]:
def get_corpus():
    filename = 'data_second.json'

    # check for presence of the file or make a new request
    if os.path.exists(filename):
        with open(filename) as json_file:
            reads = json.load(json_file)
        return pd.DataFrame(reads)
    else:
        data = make_corpus()
        with open(filename, 'w') as json_file:
            json.dump(data, json_file)
        return pd.DataFrame(data)

In [ ]:
filename = 'data_second.json'

In [ ]:
with open(filename, 'w') as json_file:
    json.dump(corpus2, json_file)

In [11]:
df_second = pd.read_json('data_second.json')

In [13]:
df_second['clean_and_lem'] = df_second.readme.apply(prepare.basic_clean).apply(prepare.lemmatize)

df_second = prepare.cut_singles(df_second)

In [15]:
tfidf = TfidfVectorizer()
tfidfs = tfidf.fit_transform(df_second.clean_and_lem.values)
X = pd.DataFrame(tfidfs.todense(), columns=tfidf.get_feature_names())

In [16]:
predictons_second = tree.predict(X)

ValueError: Number of features of the model must match the input. Model n_features is 22189 and input n_features is 22722 

In [ ]:
train['predictions'] = predictons_second

In [ ]:
model.score_your_model(train.actual, train.predictions)

In [ ]:
df_corpus2.language.value_counts()

In [18]:
df

,repo,language,readme,clean_and_lem
0,freeCodeCamp/freeCodeCamp,JavaScript,\n\n\n\n\n\nfreeCodeCamp.org's open-source cod...,freecodecamporg's opensource codebase and curr...
1,996icu/996.ICU,Rust,\n996.ICU\nPlease note that there exists NO ot...,996icu please note that there exists no other ...
2,vuejs/vue,JavaScript,\n\n\n\n\n\n\n\n\n\n\n\nSupporting Vue.js\nVue...,supporting vuejs vuejs is an mitlicensed open ...
3,facebook/react,JavaScript,\nReact · \nReact is a JavaScript library f...,react react is a javascript library for buildi...
4,tensorflow/tensorflow,C++,\n\n\n\n\n\n\nDocumentation\n\n\n\n\n\n\n\n\nT...,documentation tensorflow is an endtoend open s...
...,...,...,...,...
95,ansible/ansible,Python,\n\n \n \n \n \n \n\n\nAnsible\nAnsible is a ...,ansible ansible is a radically simple it autom...
96,kdn251/interviews,Java,\nInterviews\n\nYour personal guide to Softwar...,interview your personal guide to software engi...
97,gatsbyjs/gatsby,JavaScript,\n\n\n\n\n\n\n Gatsby v2\n\n\n⚛️ 📄 🚀\n\n\n F...,gatsby v2 fast in every way that matter gatsby...
98,opencv/opencv,C++,\nOpenCV: Open Source Computer Vision Library\...,opencv open source computer vision library res...


In [19]:
df_second

,repo,language,readme,clean_and_lem
0,antirez/redis,C,\nThis README is just a fast quick start docum...,this readme is just a fast quick start documen...
1,denoland/deno,TypeScript,\nDeno\nA secure runtime for JavaScript and Ty...,deno a secure runtime for javascript and types...
2,gohugoio/hugo,Go,\n\nA Fast and Flexible Static Site Generator ...,a fast and flexible static site generator buil...
3,ionic-team/ionic,TypeScript,\nIonic\nIonic is the open-source mobile app d...,ionic ionic is the opensource mobile app devel...
4,google/material-design-icons,CSS,\nMaterial design icons\nMaterial design icons...,material design icon material design icon is t...
...,...,...,...,...
85,Trinea/android-open-project,None,\nAndroid 开源项目分类汇总，更全更新可见 codekk.com\nOther: E...,android codekkcom other english version websit...
86,PhilJay/MPAndroidChart,Java,\n\n\n\n\n\n\n⚡️ A powerful & easy to use char...,a powerful easy to use chart library for andro...
87,tiimgreen/github-cheat-sheet,None,\nGitHub Cheat Sheet \nA collection of cool hi...,github cheat sheet a collection of cool hidden...
88,XX-net/XX-Net,Python,\nXX-Net\n中文文档 English فارسی صفحه اص...,xxnet english gaeproxyxtunnel gaeproxy xtunnel...


In [20]:
frames = [df, df_second]

df_merged = pd.concat(frames)

In [26]:
df_merged

,repo,language,readme,clean_and_lem
0,freeCodeCamp/freeCodeCamp,JavaScript,\n\n\n\n\n\nfreeCodeCamp.org's open-source cod...,freecodecamporg's opensource codebase and curr...
1,996icu/996.ICU,Rust,\n996.ICU\nPlease note that there exists NO ot...,996icu please note that there exists no other ...
2,vuejs/vue,JavaScript,\n\n\n\n\n\n\n\n\n\n\n\nSupporting Vue.js\nVue...,supporting vuejs vuejs is an mitlicensed open ...
3,facebook/react,JavaScript,\nReact · \nReact is a JavaScript library f...,react react is a javascript library for buildi...
4,tensorflow/tensorflow,C++,\n\n\n\n\n\n\nDocumentation\n\n\n\n\n\n\n\n\nT...,documentation tensorflow is an endtoend open s...
...,...,...,...,...
163,Trinea/android-open-project,None,\nAndroid 开源项目分类汇总，更全更新可见 codekk.com\nOther: E...,android codekkcom other english version websit...
164,PhilJay/MPAndroidChart,Java,\n\n\n\n\n\n\n⚡️ A powerful & easy to use char...,a powerful easy to use chart library for andro...
165,tiimgreen/github-cheat-sheet,None,\nGitHub Cheat Sheet \nA collection of cool hi...,github cheat sheet a collection of cool hidden...
166,XX-net/XX-Net,Python,\nXX-Net\n中文文档 English فارسی صفحه اص...,xxnet english gaeproxyxtunnel gaeproxy xtunnel...


In [25]:
df_merged = df_merged.drop('index',axis=1)

In [27]:
merged_json = df_merged.to_json()